In [68]:
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
import multiprocess as mp
import time
from scipy import io
from downsize_corner_finder import callCheckerboardCorners

nrows = 6   # Number of internal corner rows to be found
ncols = 8   # Number of internal corner cols to be found

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
flags = cv2.CALIB_CB_NORMALIZE_IMAGE + cv2.CALIB_CB_ACCURACY

In [69]:
def get_downsized_corners(I):

    """
    Find the checkerboard corner coordinates in downsized image
    """

    I_rows, I_cols = I.shape[0], I.shape[1]
    aspect_ratio = I_cols/I_rows

    max_downsizing = 5

    downsize_factor = 6

    for n in range(1,max_downsizing+1):
                
        manager = mp.Manager()
        return_dict = manager.dict()

        col_new = I_cols//n
        row_new = int(col_new/aspect_ratio)

        I_new = cv2.resize(I,(col_new,row_new),interpolation=cv2.INTER_AREA)
        I_new_gray = cv2.cvtColor(I_new, cv2.COLOR_BGR2GRAY)
        blurred_gray = cv2.bilateralFilter(I_new_gray, 3, 75, 75)
        # blurred_gray = cv.GaussianBlur(gray, (35,35), 0)
        # blurred_gray = cv2.equalizeHist(blurred_gray.astype(np.uint8))

        # Start checkerboard corners as a process
        p = mp.Process(target=callCheckerboardCorners(blurred_gray,(nrows,ncols),flags,return_dict))
        p.start()

        # Wait for the process to either finish or run for 10 s
        p.join(10)

        # If the thread is still alive and running, terminate
        if p.is_alive():
            print("Process killed for image size {} x {}".format(row_new,col_new))
            p.kill()
            p.join()
            continue
        else:
            p.join()
            
            print(blurred_gray.shape)
            
            downsize_factor = n
            ret = return_dict["ret"]
            corners = return_dict["corners"]
            if not ret:
                continue
            break

    if (downsize_factor == 6):
        return False, [], I, downsize_factor

    return ret, corners, blurred_gray, downsize_factor

In [70]:
def detect_corners_on_cropped_Chessboard(I,size,flags,row_min,row_max,col_min,col_max):

    """
    Try to find corners again on the cropped chessboard region
    """

    offset = 700 # 700 pixel offset
    x_min = max(col_min-offset,0)
    x_max = min(col_max+offset,I.shape[1])
    y_min = max(row_min-offset,0)
    y_max = min(row_max+offset,I.shape[0])

    cropped_region = I[y_min:y_max,x_min:x_max]

    cropped_gray = cv2.cvtColor(cropped_region, cv2.COLOR_BGR2GRAY)
    blurred_gray = cv2.bilateralFilter(cropped_gray, 3, 75, 75)

    ret, corners = cv2.findChessboardCornersSB(blurred_gray, size, flags)

    if ret:
        corners[:,:,0] = corners[:,:,0] + x_min
        corners[:,:,1] = corners[:,:,1] + y_min

    return ret, corners

In [71]:
if __name__ == '__main__':    
    dir = "/Users/jasonyuan/Desktop/Triumf Lab/Calibration Photos/Underwater/Pool Calibration Photos - June 29"

    all_corners = []
    used_files = []
    not_used = []
    n = 0

    for file in os.listdir(dir):
        # print(n)
        # print(file)
        if file == ".DS_Store":
            continue

        img = cv2.imread(dir+"/"+file)
        # gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # blurred_gray = cv2.bilateralFilter(gray,3,75,75)
        # blurred_gray = cv2.equalizeHist(blurred_gray.astype(np.uint8))
        # ret,corners = cv2.findChessboardCorners(blurred_gray, (nrows,ncols), None)
        ret,corners,im_gray,downsize_factor = get_downsized_corners(img)

        if (ret) and (downsize_factor == 1):
            used_files.append(file)
            corners2 = cv2.cornerSubPix(im_gray, corners, (11,11), (-1,-1), criteria)
            all_corners.append(np.squeeze(corners2))

        elif (ret) and (downsize_factor > 1):
            col_min = np.intc(np.min(corners[:,:,0])*downsize_factor)
            col_max = np.intc(np.max(corners[:,:,0])*downsize_factor)
            row_min = np.intc(np.min(corners[:,:,1])*downsize_factor)
            row_max = np.intc(np.max(corners[:,:,1])*downsize_factor)

            ret2, corners2 = detect_corners_on_cropped_Chessboard(img,(nrows,ncols),flags,row_min,row_max,col_min,col_max)

            if ret2:
                used_files.append(file)
                all_corners.append(np.squeeze(corners2))
            else:
                not_used = not_used + [file]

        else:
            not_used = not_used + [file]

        n += 1

(6336, 9504)
(3168, 4752)
(6336, 9504)
(6336, 9504)
(3168, 4752)
(6336, 9504)
(3168, 4752)
(6336, 9504)
(3168, 4752)
(2112, 3168)
(6336, 9504)
(6336, 9504)
(3168, 4752)
(6336, 9504)
(6336, 9504)
(6336, 9504)
(3168, 4752)
(6336, 9504)
(6336, 9504)
(3168, 4752)
(6336, 9504)
(6336, 9504)
(3168, 4752)
(6336, 9504)
(3168, 4752)
(2112, 3168)
(6336, 9504)
(6336, 9504)
(6336, 9504)
(6336, 9504)
(3168, 4752)
(6336, 9504)
(6336, 9504)
(6336, 9504)
(3168, 4752)
(6336, 9504)
(6336, 9504)
(3168, 4752)
(2112, 3168)
(6336, 9504)
(3168, 4752)
(6336, 9504)
(6336, 9504)
(6336, 9504)
(3168, 4752)
(2112, 3168)
(6336, 9504)
(6336, 9504)
(6336, 9504)
(3168, 4752)
(6336, 9504)
(6336, 9504)
(6336, 9504)
(3168, 4752)
(2112, 3168)
(6336, 9504)
(3168, 4752)
(6336, 9504)
(6336, 9504)
(6336, 9504)
(6336, 9504)
(6336, 9504)
(6336, 9504)
(3168, 4752)
(6336, 9504)
(3168, 4752)
(6336, 9504)
(3168, 4752)
(2112, 3168)
(6336, 9504)
(6336, 9504)
(6336, 9504)
(6336, 9504)
(6336, 9504)
(6336, 9504)
(3168, 4752)
(6336, 9504)

In [72]:
df = pd.DataFrame({"used_files": used_files, "corners": all_corners})
df.to_csv("/Users/jasonyuan/Desktop/used_files.csv")

np.savetxt("/Users/jasonyuan/Desktop/all_corners_{}.csv".format(np.array(all_corners).shape), np.array(all_corners).flatten(), delimiter=",")
io.savemat("/Users/jasonyuan/Desktop/all_corners.mat", {"corners": np.array(all_corners).transpose(1,2,0)})


In [74]:
np.array(all_corners).transpose(1,2,0)

array([[[5004.8384, 5553.6025, 2367.8726, ..., 3427.044 , 4944.4946,
         4485.4395],
        [1820.5927, 2265.539 , 4037.2598, ..., 2370.511 , 2275.2178,
         3909.5852]],

       [[5006.944 , 5556.64  , 2316.3926, ..., 3404.737 , 4957.47  ,
         4475.6914],
        [1623.5011, 2093.0312, 3856.731 , ..., 2202.9595, 2559.0825,
         3797.8594]],

       [[5014.5312, 5559.4233, 2276.0693, ..., 3386.2458, 4970.3267,
         4465.513 ],
        [1430.8939, 1918.7665, 3695.6953, ..., 2038.4462, 2832.1743,
         3684.899 ]],

       ...,

       [[6482.2603, 6871.3877, 3301.41  , ..., 4573.52  , 3611.7324,
         5298.419 ],
        [1252.3322, 1812.4526, 3514.906 , ..., 1758.2068, 2954.2166,
         3524.745 ]],

       [[6476.062 , 6881.694 , 3267.805 , ..., 4564.3584, 3646.7358,
         5292.3545],
        [1064.7744, 1620.6614, 3291.0886, ..., 1589.7227, 3147.1462,
         3405.1626]],

       [[6466.8745, 6889.157 , 3229.9563, ..., 4557.608 , 3680.766 ,
        

In [77]:
np.array(all_corners).shape


(247, 48, 2)

In [84]:
with open("/Users/jasonyuan/Desktop/not_used.txt",'w') as f:
    for file in not_used:
        f.write(file+"\n")